In [9]:
from mesa import Agent
from mesa import Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random

In [10]:
    infection_rate = 0.1 # # probability of an infected bacteria transfering ARG to 
                          # adjcent susceptible bacteria
    recovery_rate = 0.1 # probability of a infected bacteria to recover
    arg_rate = 0.1 # probability of a bacteria to have ARG intitially

# Define agent Bacteria

In [11]:
class Bacteria(Agent):
    def __init__(self, unique_id,model,state=0,age=0,volume=0.1,tax='bac',gene_arg=None,mge=None):
        super().__init__(unique_id, model)
        self.unique_id = unique_id
        self.model = model
        self.state = state # 0 susceptible; 1 infected; 2 recovered
        self.age=age
        self.volume = volume
        self.tax = tax
        self.gene_arg = gene_arg
        self.mge = mge

    def step(self):
        self.random_move()
        self.divide()
        self.grow()
        self.interact()


    def random_move(self):
        ''' Bacteria can randomly move to nearby grids'''
        neighbor_pos = self.model.grid.get_neighborhood(self.pos, moore=True,include_center=False)
        new_pos = random.choice(neighbor_pos)
        self.model.grid.move_agent(self,new_pos)

    def grow(self):
        self.age += 1
        self.volume = 0.1 + self.age*0.005

    def divide(self):
        if self.volume == 0.2: # divide when the bacteria reaches 20 time steps
            self.age = 0
            new_born = Bacteria(self.model.num_agents+1,self.model,
            self.state,0,0.1,self.tax,self.gene_arg,self.mge) # inherit the genes from parent cell
            self.model.grid.place_agent(new_born,self.pos)
            self.model.schedule.add(new_born)
            self.model.num_agents += 1

    def interact(self):
        ''' interact with bacteria located in the same grid'''
        # search the agents located in the same grids

        if self.state == 1:
            x, y = self.pos
            this_cell = self.model.grid.get_cell_list_contents([self.pos])
            susceptible_bacteria = [obj for obj in this_cell if obj.state==0]
            # randomly infected a susceptible_bacteria
            if len(susceptible_bacteria)>0:
                for each in susceptible_bacteria:
                    # randomly sample a number p, if p<infection rate, infection happens
                    p = random.uniform(0,1)
                    if p< infection_rate:
                        each.state = 1
                        each.gene_arg = self.gene_arg

            # after infection, there is a chance to recovered
            p = random.uniform(0,1)
            if p<recovery_rate:
                self.state = 2

# Define model Bacteria_Colony

In [12]:
class Bacteria_Colony(Model):
    def __init__(self,N=100,width=10,height=10):
        super().__init__()
        self.num_agents = N
        self.width =width
        self.height = height
        self.grid = MultiGrid(width,height,True)

        self.schedule = RandomActivation(self)

        # DataCollector
        self.datacollector = DataCollector(
                # model level data
                model_reporters={
                 "PopSize":"N",
                 "Susceptible":lambda m: m.count_head(0),
                 "Infected": lambda m: m.count_head(1),
                 "Recovered": lambda m: m.count_head(2)},
                
                # agent level data
                agent_reporters={
                "State":"state",
                    "Age":"age",
                    "Volume":"volume",
                    "Taxonomy":"tax",
                    "ARG":"gene_arg",
                    "MGE":"mge"               
                }
                
        
        
        )
                 # m corresponds to the model, which is the argument of the function

        # create and assign agents
        coords_list = [(x,y) for x in range(width) for y in range(height)]

        for i in range(self.num_agents):
            # randomly assign the agent to a cell
            coords = random.choice(coords_list)
            #print(coords)
            p = random.uniform(0,1)
            if p<arg_rate:
                state = 1
            else:
                state = 0
            a = Bacteria(i,self,state)
            self.grid.place_agent(a, coords)
            self.schedule.add(a)

        #self.running=True
        #self.datacollector.collect(self)

    def step(self):      
        self.datacollector.collect(self) # collect model level data during each step
        self.schedule.step()
        #print(self.num_agents)
        #print(self.count_head(0))
        #print(self.count_head(1))
        #print(self.count_head(2))

    def run_model(self,step_count=200):
        for i in range(step_count):
            #print("The model is currently running step %d" % (i))
            self.step()

    def count_head(self,type):
        # type 0: Susceptible, 1: Infected, 2: Recovered
        count = 0;
        tmp = self.schedule._agents
        for each in tmp.values():
            if each.state == type:
                count+=1
        return count



# Test Zone

In [13]:
model = Bacteria_Colony()

In [14]:
model.run_model()

In [15]:
agents = model.datacollector.get_agent_vars_dataframe()

In [16]:
agents

State  Age  Volume Taxonomy   ARG   MGE
Step AgentID                                         
0    0            1    0    0.10      bac  None  None
     1            1    0    0.10      bac  None  None
     2            1    0    0.10      bac  None  None
     3            0    0    0.10      bac  None  None
     4            0    0    0.10      bac  None  None
...             ...  ...     ...      ...   ...   ...
199  51196        2   10    0.15      bac  None  None
     51197        2   10    0.15      bac  None  None
     51198        2   10    0.15      bac  None  None
     51199        2   10    0.15      bac  None  None
     51200        2   10    0.15      bac  None  None

[1815600 rows x 6 columns]